In [82]:
%config Completer.use_jedi = False
from pymongo import MongoClient
from dotenv import load_dotenv
from tqdm import tqdm
import numpy as np
import pandas as pd
import datetime as dt
import praw
import os
import time

load_dotenv()

reddit = praw.Reddit(
    client_id = os.environ.get('REDDIT_CLIENT_ID_2'),
    client_secret = os.environ.get('REDDIT_CLIENT_SECRET_2'),
    user_agent = os.environ.get('REDDIT_USER_AGENT_2'),
    username = os.environ.get('REDDIT_USERNAME_2'),
    password = os.environ.get('REDDIT_PASSWORD_2')
)

assert reddit is not None and not reddit.read_only

client = MongoClient('localhost', 27017)
all_comments = client.resilient.all

In [140]:
df = pd.read_pickle('../../removed/PeopleFuckingDying.pkl')

In [141]:
df[df['removed'] == True]

,id,retrieved_utc,removed
60,i1vm0t7,1649294657,True
212,i1ybxg2,1649294657,True
217,i1ygkj8,1649294657,True
241,i1z0w7m,1649294657,True
394,i225i5s,1649294658,True
...,...,...,...
4312,i3i9pd6,1649294677,True
4359,i3if3iq,1649294677,True
4372,i3iiish,1649294677,True
4379,i3ijmi9,1649294677,True


In [65]:
sample = pd.DataFrame(all_comments.aggregate([
    {'$sample': {'size': 5_000}},
    {'$project': {'id': 1, 'created_utc': 1, 'permalink': 1, 'body': 1}}
], allowDiskUse=True))

sample['fullname'] = 't1_' + sample['id']

sample

,_id,id,body,permalink,created_utc,fullname
0,623bd0dbbe7259265328e5e4,i1vmbn8,Sure do…,/r/slutsofsnapchat/comments/tlmqju/do_men_like...,1.648087e+09,t1_i1vmbn8
1,623f8b3fbe725926535263fb,i28ohgz,I found a 2000 and I will buy it but I wanna k...,/r/vitahacks/comments/tnoqxs/ps3_slim_cfw_vs_v...,1.648331e+09,t1_i28ohgz
2,623dc782be72592653c05169,i22afkc,A necessary evil,/r/facepalm/comments/tnk4ek/husband_of_the_yea...,1.648215e+09,t1_i22afkc
3,623e153cbe72592653dd3b8a,i23o82j,Upvoted you! Please return on my posts?,/r/FreeKarma4U/comments/tnp0nd/upvote_my_comme...,1.648235e+09,t1_i23o82j
4,624517b8be725926530b3541,i2ssnt2,Lol bill and Melinda gates foundation,/r/ShitLiberalsSay/comments/tsou6m/this_is_low...,1.648694e+09,t1_i2ssnt2
...,...,...,...,...,...,...
4995,623bcce5be7259265327c877,i1vkndu,"Fuck, thats hot",/r/gonewild/comments/tlsgk4/after_8_weeks_of_p...,1.648086e+09,t1_i1vkndu
4996,623bb8e2be7259265320ceb3,i1va4jm,Thanks for the suggestion. I’ve spent the last...,/r/tacticalgear/comments/tlmdns/i_understand_t...,1.648081e+09,t1_i1va4jm
4997,623dcb81be72592653c2761a,i22dojs,You dash in NYC? The traffic must be a nightma...,/r/doordash_drivers/comments/tnarw5/possible_s...,1.648217e+09,t1_i22dojs
4998,623c5633be725926534a7534,i1wzwlc,Redreader app installieren!,/r/FragReddit/comments/tm21uq/nutzt_ihr_reddit...,1.648120e+09,t1_i1wzwlc


# 5,000 Comments, 100 Batches

In [67]:
BATCH_SIZE = 100

results = {}

for i in tqdm(range(0, len(sample), BATCH_SIZE)):
    batch = sample[i : i + BATCH_SIZE]['fullname'].values.tolist()
    
    for comment in reddit.info(fullnames=batch):
        results[comment.fullname] = comment.body == '[removed]'
        
results = pd.DataFrame.from_dict(results, orient='index')

sample.join(results, on='fullname')

100%|██████████| 50/50 [00:35<00:00,  1.41it/s]


,_id,id,body,permalink,created_utc,fullname,0
0,623bd0dbbe7259265328e5e4,i1vmbn8,Sure do…,/r/slutsofsnapchat/comments/tlmqju/do_men_like...,1.648087e+09,t1_i1vmbn8,False
1,623f8b3fbe725926535263fb,i28ohgz,I found a 2000 and I will buy it but I wanna k...,/r/vitahacks/comments/tnoqxs/ps3_slim_cfw_vs_v...,1.648331e+09,t1_i28ohgz,False
2,623dc782be72592653c05169,i22afkc,A necessary evil,/r/facepalm/comments/tnk4ek/husband_of_the_yea...,1.648215e+09,t1_i22afkc,False
3,623e153cbe72592653dd3b8a,i23o82j,Upvoted you! Please return on my posts?,/r/FreeKarma4U/comments/tnp0nd/upvote_my_comme...,1.648235e+09,t1_i23o82j,False
4,624517b8be725926530b3541,i2ssnt2,Lol bill and Melinda gates foundation,/r/ShitLiberalsSay/comments/tsou6m/this_is_low...,1.648694e+09,t1_i2ssnt2,False
...,...,...,...,...,...,...,...
4995,623bcce5be7259265327c877,i1vkndu,"Fuck, thats hot",/r/gonewild/comments/tlsgk4/after_8_weeks_of_p...,1.648086e+09,t1_i1vkndu,False
4996,623bb8e2be7259265320ceb3,i1va4jm,Thanks for the suggestion. I’ve spent the last...,/r/tacticalgear/comments/tlmdns/i_understand_t...,1.648081e+09,t1_i1va4jm,False
4997,623dcb81be72592653c2761a,i22dojs,You dash in NYC? The traffic must be a nightma...,/r/doordash_drivers/comments/tnarw5/possible_s...,1.648217e+09,t1_i22dojs,False
4998,623c5633be725926534a7534,i1wzwlc,Redreader app installieren!,/r/FragReddit/comments/tm21uq/nutzt_ihr_reddit...,1.648120e+09,t1_i1wzwlc,False


In [68]:
BATCH_SIZE = 500

results = {}

for i in tqdm(range(0, len(sample), BATCH_SIZE)):
    batch = sample[i : i + BATCH_SIZE]['fullname'].values.tolist()
    
    for comment in reddit.info(fullnames=batch):
        results[comment.fullname] = comment.body == '[removed]'
        
results = pd.DataFrame.from_dict(results, orient='index')

sample.join(results, on='fullname')

100%|██████████| 10/10 [00:34<00:00,  3.40s/it]


,_id,id,body,permalink,created_utc,fullname,0
0,623bd0dbbe7259265328e5e4,i1vmbn8,Sure do…,/r/slutsofsnapchat/comments/tlmqju/do_men_like...,1.648087e+09,t1_i1vmbn8,False
1,623f8b3fbe725926535263fb,i28ohgz,I found a 2000 and I will buy it but I wanna k...,/r/vitahacks/comments/tnoqxs/ps3_slim_cfw_vs_v...,1.648331e+09,t1_i28ohgz,False
2,623dc782be72592653c05169,i22afkc,A necessary evil,/r/facepalm/comments/tnk4ek/husband_of_the_yea...,1.648215e+09,t1_i22afkc,False
3,623e153cbe72592653dd3b8a,i23o82j,Upvoted you! Please return on my posts?,/r/FreeKarma4U/comments/tnp0nd/upvote_my_comme...,1.648235e+09,t1_i23o82j,False
4,624517b8be725926530b3541,i2ssnt2,Lol bill and Melinda gates foundation,/r/ShitLiberalsSay/comments/tsou6m/this_is_low...,1.648694e+09,t1_i2ssnt2,False
...,...,...,...,...,...,...,...
4995,623bcce5be7259265327c877,i1vkndu,"Fuck, thats hot",/r/gonewild/comments/tlsgk4/after_8_weeks_of_p...,1.648086e+09,t1_i1vkndu,False
4996,623bb8e2be7259265320ceb3,i1va4jm,Thanks for the suggestion. I’ve spent the last...,/r/tacticalgear/comments/tlmdns/i_understand_t...,1.648081e+09,t1_i1va4jm,False
4997,623dcb81be72592653c2761a,i22dojs,You dash in NYC? The traffic must be a nightma...,/r/doordash_drivers/comments/tnarw5/possible_s...,1.648217e+09,t1_i22dojs,False
4998,623c5633be725926534a7534,i1wzwlc,Redreader app installieren!,/r/FragReddit/comments/tm21uq/nutzt_ihr_reddit...,1.648120e+09,t1_i1wzwlc,False


In [70]:
BATCH_SIZE = 1_000

results = {}

for i in tqdm(range(0, len(sample), BATCH_SIZE)):
    batch = sample[i : i + BATCH_SIZE]['fullname'].values.tolist()
    
    for comment in reddit.info(fullnames=batch):
        results[comment.fullname] = comment.body == '[removed]'
        
results = pd.DataFrame.from_dict(results, orient='index')

sample = sample.join(results, on='fullname')

100%|██████████| 5/5 [00:34<00:00,  6.84s/it]


In [99]:
sample[(sample[0] != True) & (sample[0] != False)]

,_id,id,body,permalink,created_utc,fullname,0
80,623d4aa1be72592653a1c045,i210i6i,Mmmmmm that would be wonderful sexy lady,/r/butt/comments/tdne2y/let_me_bounce_on_your_...,1.648183e+09,t1_i210i6i,NaN
87,623bd4f3be725926532b8a11,i1vq7ej,🧃 - FRE£ 0NLYFAN$ LEAK$ www.t.me/mgdpz/,/r/xxxycelebs/comments/tlv04o/_/i1vq7ej/,1.648088e+09,t1_i1vq7ej,NaN
399,624588d1be7259265327e92a,i2tz2ot,There is alwalys time for that! Lol,/r/BiCuriousGuysChat/comments/tsx38g/wish_i_ha...,1.648724e+09,t1_i2tz2ot,NaN
556,623bd92abe725926532c557e,i1vrdzw,Mmm yes please,/r/Wrestling_Babes/comments/tkkwzx/jessica_mck...,1.648089e+09,t1_i1vrdzw,NaN
899,623ba8eabe7259265319cf03,i1uzaps,Me too count me in,/r/Zoomsexhookup/comments/tl033a/i_am_up_for_t...,1.648076e+09,t1_i1uzaps,NaN
1166,623bd929be725926532b961d,i1vqanx,"Hey, thank you for participating in this new c...",/r/FanslyModels/comments/tlv2vw/so_satisfying/...,1.648088e+09,t1_i1vqanx,NaN
1383,623bb8e1be725926531f747c,i1v8352,"if they were ""acting"", they deserve an Academy...",/r/celebsnaked/comments/tlltqi/lea_seydoux_ade...,1.648080e+09,t1_i1v8352,NaN
1685,62449988be72592653dcbe2c,i2qq4ue,Dm,/r/tradenudefriends/comments/trn6o0/18_gf_to_t...,1.648663e+09,t1_i2qq4ue,NaN
1716,623bb4e5be725926531eafeb,i1v6wkl,https://discord.gg/nKkGTeRT,/r/RPfr/comments/tlcor6/venez_me_dominer/i1v6wkl/,1.648080e+09,t1_i1v6wkl,NaN
2461,623bb4e5be725926531e7718,i1v6kgv,dont miss out NEW onIyfans of the baddest thot...,/r/kaylalebombb/comments/tlmetv/hot4lexi_new_u...,1.648080e+09,t1_i1v6kgv,NaN


In [100]:
BATCH_SIZE = 1_000

results = {}

for i in range(0, len(sample), BATCH_SIZE):
    batch = sample[i : i + BATCH_SIZE]['fullname'].values.tolist()
    
    for comment in tqdm(reddit.info(fullnames=batch)):
        results[comment.fullname] = comment.body == '[removed]'
        
    break
        
results = pd.DataFrame.from_dict(results, orient='index')

sample = sample.join(results, on='fullname')

995it [00:08, 123.62it/s]


ValueError: columns overlap but no suffix specified: Index([0], dtype='object')

In [135]:
list(reddit.info(fullnames=['t1_i1vf0bc']))

[Comment(id='i1vf0bc')]